In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Model

import numpy as np

# Lee el archivo de texto y crea una lista de oraciones
with open('los3.txt', 'r') as file:
    data = file.read().replace('\n', '')

print(data)
sentences = data.split('.')
print(sentences)    
# Tokeniza las oraciones
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences)
total_words = len(tokenizer.word_index) + 1

# Crea secuencias de palabras para el entrenamiento del modelo
input_sequences = []
for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Obtiene la longitud máxima de secuencia y rellena las secuencias para que tengan la misma longitud
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Crea los datos de entrada y salida para el modelo
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Crea el modelo de red neuronal
input_layer = Input(shape=(max_sequence_len-1,))
embedding_layer = Embedding(total_words, 10, input_length=max_sequence_len-1)(input_layer)
lstm_layer = LSTM(100)(embedding_layer)
dropout_layer = Dropout(0.1)(lstm_layer)
output_layer = Dense(total_words, activation='softmax')(dropout_layer)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrena el modelo
model.fit(xs, ys, epochs=100, verbose=1)


Había una vez tres cerditos que eran hermanos y se fueron por el mundo a conseguir fortuna. El más grande les dijo a sus hermanos que sería bueno que se pusieran a construir sus propias casas para estar protegidos. A los otros dos les pareció una buena idea, y se pusieron manos a la obra, cada uno construyó su casita.  - La mía será de paja - dijo el más pequeño-, la paja es blanda y se puede sujetar con facilidad. Terminaré muy pronto y podré ir a jugar. El hermano mediano decidió que su casa sería de madera:  - Puedo encontrar un montón de madera por los alrededores - explicó a sus hermanos, - Construiré mi casa en un santiamén con todos estos troncos y me iré también a jugar.  Cuando las tres casitas estuvieron terminadas, los cerditos cantaban y bailaban en la puerta, felices por haber acabado con el problema:  -¡Quién teme al Lobo Feroz, al Lobo, al Lobo!  - ¡Quién teme al Lobo Feroz, al Lobo Feroz! Detrás de un árbol grande apareció el lobo, rugiendo de hambre y gritando:  - Cerd

In [2]:
import re

In [3]:
# Crear un diccionario de caracteres únicos y asignar un número a cada uno
char_to_idx = {char: idx for idx, char in enumerate(sorted(set(data)))}

# Crear el diccionario inverso
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

In [4]:
# Longitud de secuencia para la entrada del modelo
seq_length = 100

In [5]:
def generate_text(model, start_string, num_generate=10, temperature=0.1):
    print(start_string)
    string = start_string.lower()
    print(string)
    string = re.sub(r'[^a-zñáéíóúü]', ' ', string)
    input_eval = [char_to_idx[char] for char in string]
    if len(input_eval) < max_sequence_len-1:
        input_eval = tf.pad(input_eval, [(0, max_sequence_len-1 - len(input_eval))], mode='CONSTANT', constant_values=0)
    else:
        input_eval = np.roll(input_eval, max_sequence_len-1-(len(input_eval)-1))
    input_eval = tf.reshape(input_eval, (1, max_sequence_len-1))

    # Crear una lista vacía para almacenar el texto generado
    text_generated = []

    # Establecer la temperatura
    model.reset_states()
    temperature = temperature

    for i in range(num_generate):
        # Predecir el siguiente carácter utilizando el modelo
        predictions = model(input_eval)
        predictions = predictions / temperature
        
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        predicted_id = predicted_id if predicted_id < len(idx_to_char) else len(idx_to_char) - 1

        # Añadir el carácter generado a la lista de texto generado
        text_generated.append(idx_to_char[predicted_id])

        # Utilizar el carácter generado como entrada para la siguiente predicción
        input_eval = tf.concat([input_eval[:, 1:], tf.expand_dims([predicted_id], 1)], axis=-1)

    return ( ''.join(text_generated))





In [6]:
# Generar texto nuevo
generated_text = generate_text(model, start_string='Los Tres Cerditos', num_generate=100, temperature=1.5)
print()
# Imprimir el texto generado
print(generated_text)

Los Tres Cerditos
los tres cerditos

úúúúúú úúúYLúúDúQúúúúLúúúúúúAúDúúúúúóúúHEúúúúúbúúúúúúúúxúúúiúúúúúújúúúúíúúúNLúúúúúúúúpúúúúúúúúúúúúYú
